In [4]:
import pandas as pd

In [33]:
txt = open("ar_cations.txt", "r")

In [34]:
lines = txt.readlines()

In [54]:
lines[555].split('\t')

['1507563902_6ec8d5d822.jpg#0', 'طفل مقلوب فوق الترامبولين\n']

In [57]:
image = lines[555].split('\t')[0].split("#")[0]; image

'1507563902_6ec8d5d822.jpg'

In [56]:
line = lines[0].split('\t')[1].strip(); line

'طفلة صغيرة تتسلق إلى مسرح خشبي'

In [44]:
import pyarabic.araby as araby

In [45]:
araby.tokenize(line)

['طفلة', 'صغيرة', 'تتسلق', 'إلى', 'مسرح', 'خشبي']

In [58]:
images, captions = [], []
for line in lines:
    images.append(line.split('\t')[0].split("#")[0])
    captions.append(line.split('\t')[1].strip())

In [59]:
images[:4], captions[:4]

(['1000268201_693b08cb0e.jpg',
  '1001773457_577c3a7d70.jpg',
  '1002674143_1b742ab4b8.jpg',
  '1003163366_44323f5815.jpg'],
 ['طفلة صغيرة تتسلق إلى مسرح خشبي',
  'كلب أسود وكلب ثلاثي الألوان يلعبان مع بعضهما البعض على الطريق',
  'فتاة صغيرة مغطاة بالطلاء تجلس أمام قوس قزح',
  'رجل يجلس على مقعد بينما يجلس كلبه بجانبه'])

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import spacy

from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [62]:
class Vocabulary:
    def __init__(self, freq_threshold=2):
        self.freq_threshold = freq_threshold
        self.itos = {0:"<PAD>", 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"}
        self.stoi = {"<PAD>":0, "<SOS>":1, "<EOS>":2, "<UNK>":3}
        
    def __len__(self):
        return len(self.itos)
    
    @staticmethod
    def tokenize_en(text):
        return [tok.lower() for tok in araby.tokenize(text)]
    
    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4
        
        for sentence in sentence_list:
            for word in self.tokenize_en(sentence):
                if word not in frequencies:
                    frequencies[word] = 1
                else:
                    frequencies[word] += 1
                
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1
    def numericalize(self, text):
        tokenizd_text = self.tokenize_en(text)
        
        return [self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] 
                for token in tokenizd_text]

In [63]:
vocab = Vocabulary()
vocab.build_vocabulary(captions)

In [66]:
[vocab.itos[i] for i in vocab.numericalize(captions[1])]

['كلب',
 'أسود',
 'وكلب',
 'ثلاثي',
 'الألوان',
 'يلعبان',
 'مع',
 'بعضهما',
 'البعض',
 'على',
 'الطريق']

In [67]:
captions[1]

'كلب أسود وكلب ثلاثي الألوان يلعبان مع بعضهما البعض على الطريق'

In [70]:
class FlickrDataset(Dataset):
    def __init__(self, root_dir, transform=None, freq_threshold=2):
        self.root_dir = root_dir
        self.transform = transform
        
        self.imgs = images
        self.captions = captions
        
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]
        img = Image.open(self.root_dir+'/Images/'+img_id).convert('RGB')
        
        if self.transform is not None:
            img = self.transform(img)
        
        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])
        
        return img, torch.tensor(numericalized_caption)

In [71]:
ds = FlickrDataset(".")

In [72]:
plt.imshow(ds[0][0]);
' '.join([ds.vocab.itos[i.item()] for i in ds[0][1]])

FileNotFoundError: [Errno 2] No such file or directory: './Images/1000268201_693b08cb0e.jpg'